# MAAP Tutorial

### `maap-py`
Click on the `MAAP` button in the Jupyter Toolbar to import the `maap-py` library.<br>
![](./images/insert_defaults_button.png)<br><br>
We will only need the first 2 lines:

In [2]:
from maap.maap import MAAP
maap = MAAP()

# json and xml libraries to parse some responses
import json
import xml.etree.ElementTree as ET

## EarthData Search Utilities

In [ ]:
# maap.searchGranule
# maap.getCallFromEarthDataQuery
# maap.getCallFromCmrUri
# maap.searchCollection

## MAS Utilities

### List Algorithms

In [6]:
response = maap.listAlgorithms()
json_response = json.loads(response.text)
print(json_response)

{'code': 200, 'message': 'success', 'algorithms': [{'type': 'hello-world-output_ubuntu', 'version': 'master'}, {'type': 'agb_jaxa_demo_apr20_ubuntu', 'version': 'jaxa_analysis'}, {'type': 'agb_jaxa_demo_metrics_ubuntu', 'version': 'jaxa_analysis'}]}


### Describe Algorithm

In [46]:
# helper to parse out algorithm parameters for execute, describe
def getParams(node):
	tag = node.tag[node.tag.index('}')+1:]
	if tag in ['Title','Identifier']:
		return (tag,node.text)
	elif tag == 'LiteralData':
		return (node[1][1].tag.split('}')[-1],list(node[1][1].attrib.values())[0].split(':')[-1])
	else:
		return (tag,[getParams(e) for e in node])

In [18]:
response = maap.describeAlgorithm('agb_jaxa_demo_metrics_ubuntu:jaxa_analysis')
rt = ET.fromstring(response.text)
attrib = [getParams(e) for e in rt[0][0]]
for e in attrib:
    print(e)

('Title', 'Algorithm: agb_jaxa_demo_metrics_ubuntu ; Version: jaxa_analysis')
('Identifier', 'agb_jaxa_demo_metrics_ubuntu:jaxa_analysis')
('Input', [('Title', 'granule'), ('Identifier', 'granule'), ('DataType', 'string')])
('Input', [('Title', 'clip_upper_bound'), ('Identifier', 'clip_upper_bound'), ('DataType', 'string')])
('Output', [])


In [ ]:
# maap.registerAlgorithm
# maap.deleteAlgorithm

## DPS Utilities

### Submit Job

In [3]:
params = {}
params['algo_id'] = 'agb_jaxa_demo_metrics_ubuntu'
params['version'] = 'jaxa_analysis'
params['inputs'] = 'granule,clip_upper_bound'
params['granule'] = 'https://cumulus-map-internal.s3.amazonaws.com/file-staging/nasa-map/Global_PALSAR2_PALSAR_Mosiac___1/N25E030_17_MOS_F02DAR.tar.gz'
params['clip_upper_bound'] = '-15'
params['username'] = 'eyam'
response = maap.submitJob(**params)
print(response)

{'status': 'success', 'http_status_code': 200, 'job_id': '1b305fe3-e078-4da4-8036-c71878586bdc'}


### List Jobs

In [28]:
response = maap.listJobs('eyam')
json_response = json.loads(response.text)
json_response

{'code': 200,
 'jobs': ["['7e31829d-cb84-4b0e-9cda-ed0799a972ba', 'job-started', 'job-agb_jaxa_demo_metrics_ubuntu:jaxa_analysis', [{'name': 'granule', 'destination': 'localize', 'value': 'https://cumulus-map-internal.s3.amazonaws.com/file-staging/nasa-map/Global_PALSAR2_PALSAR_Mosiac___1/N25E030_17_MOS_F02DAR.tar.gz'}, {'name': 'clip_upper_bound', 'destination': 'positional', 'value': -15}]]",
  "['76f655a8-530c-4fbd-9e5a-f3446807e172', 'job-completed', 'job-agb_jaxa_demo_metrics_ubuntu:jaxa_analysis', [{'name': 'granule', 'destination': 'localize', 'value': 'https://cumulus-map-internal.s3.amazonaws.com/file-staging/nasa-map/Global_PALSAR2_PALSAR_Mosiac___1/N25E030_17_MOS_F02DAR.tar.gz'}, {'name': 'clip_upper_bound', 'destination': 'positional', 'value': -15}]]",
  "['b88b201e-d7cd-497f-be27-c014e7618a1c', 'job-queued', 'job-agb_jaxa_demo_metrics_ubuntu:jaxa_analysis', [{'name': 'granule', 'destination': 'localize', 'value': 'https://cumulus-map-internal.s3.amazonaws.com/file-staging

### Job Status

In [42]:
response = maap.getJobStatus('7e31829d-cb84-4b0e-9cda-ed0799a972ba')
if response.status_code == 200:
    rt = ET.fromstring(response.text)
    job_id = rt[0].text
    job_status = rt[1].text
    print(job_id)
    print(job_status)

7e31829d-cb84-4b0e-9cda-ed0799a972ba
Succeeded


### Job Result

In [47]:
# helpers to parse results XML

def getProds(node):
	tag = node.tag[node.tag.index('}')+1:]
	if tag in ['JobID']:
		return (tag,node.text)
	elif tag == 'Output':
		return (tag,[loc.text for loc in node])
	else:
		return (tag,[getProds(e) for e in node])

def parse_results(rt):
    job_id = rt[0].text
    product_name = rt[1].attrib['id']
    prods = rt[1]
    p = getProds(prods)
    url_lst = p[1]
    return {'job_id':job_id, 'product_name':product_name, 'locations':url_lst}


In [45]:
response = maap.getJobResult('7e31829d-cb84-4b0e-9cda-ed0799a972ba')
if response.status_code == 200:
    rt = ET.fromstring(response.text)
    results = parse_results(rt)
    print(results)

else:
    print(response.text)

{'job_id': '7e31829d-cb84-4b0e-9cda-ed0799a972ba', 'product_name': 'output-2020-08-27T18:55:08.526237', 'locations': ['http://maap-dev-dataset.s3-website.amazonaws.com/null/dps_output/agb_jaxa_demo_metrics_ubuntu/jaxa_analysis/2020/08/27/18/55/08/526237', 's3://s3.amazonaws.com:80/maap-dev-dataset/null/dps_output/agb_jaxa_demo_metrics_ubuntu/jaxa_analysis/2020/08/27/18/55/08/526237', 'https://s3.console.aws.amazon.com/s3/buckets/maap-dev-dataset/null/dps_output/agb_jaxa_demo_metrics_ubuntu/jaxa_analysis/2020/08/27/18/55/08/526237/?region=us-east-1&tab=overview']}


### Job Metrics

In [51]:
def parse_metrics(node):
    metrics = {}
    for n in node:
        metrics[n.tag] = n.text
        
    return metrics

In [49]:
response = maap.getJobMetrics('7e31829d-cb84-4b0e-9cda-ed0799a972ba')

In [52]:
rt = ET.fromstring(response.text)
parse_metrics(rt)

{'machine_type': 't3a.large',
 'architecture': 'x86_64',
 'machine_memory_size': '7.56 GB',
 'directory_size': '8493764608',
 'operating_system': 'CentOS',
 'job_start_time': '2020-08-27T18:49:53.050557Z',
 'job_end_time': '2020-08-27T18:55:11.193153Z',
 'job_duration_seconds': '318.142596',
 'cpu_usage': '288082375909',
 'cache_usage': '3982348288',
 'mem_usage': '3983478784',
 'max_mem_usage': '6185947136',
 'swap_usage': '0',
 'read_io_stats': '4096',
 'write_io_stats': '14848',
 'sync_io_stats': '14848',
 'async_io_stats': '4096',
 'total_io_stats': '18944'}

### Dismiss Job

In [ ]:
response = maap.dismissJob('b88b201e-d7cd-497f-be27-c014e7618a1c')

In [41]:
if response.text == 200:
    rt = ET.fromstring(response.text)
    job_id = rt[0].text
    job_status = rt[1].text
    print(job_id)
    print(job_status)
else:
    print(response.text)

'<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>504 Gateway Timeout</title>\n</head><body>\n<h1>Gateway Timeout</h1>\n<p>The gateway did not receive a timely response\nfrom the upstream server or application.</p>\n</body></html>\n'

### Delete Job

In [55]:
response = maap.deleteJob('b88b201e-d7cd-497f-be27-c014e7618a1c')

In [56]:
if response.status_code == 200:
    rt = ET.fromstring(response.text)
    job_id = rt[0].text
    job_status = rt[1].text
    print(job_id)
    print(job_status)
else:
    print(response.text)

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>504 Gateway Timeout</title>
</head><body>
<h1>Gateway Timeout</h1>
<p>The gateway did not receive a timely response
from the upstream server or application.</p>
</body></html>



## Other Utilities

In [ ]:
# maap.uploadFiles
# maap.executeQuery